In [2]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

In [3]:
%load_ext nb_black

<IPython.core.display.Javascript object>

# eda_final_project

## Duomenys

Duomenų šaltinis https://www.registrucentras.lt/p/1094. 

1. Parsisiųsti 2020 ir 2021 įmonių “Juridinių asmenų pateikti finansinės atskaitomybės dokumentai – balanso ataskaitos” ir “Juridinių asmenų pateikti finansinės atskaitomybės dokumentai – pelno (nuostolių) ataskaitos” duomenis. 

1. Apskaičiuoti vidutinias reikšmes “pelnas_pries_apmokestinima” ir “nuosavas_kapitalas” laukų kiekvienai juridinio asmens formai (Uždaroji akcinė bendrovė, Mažoji bendrija, etc.) už visą laikotarpį. 

2. Atrinkti įmones, kurių “nuosavas_kapitalas” buvo didesnis negu nuosavo kapitalo mediana 2020 ir 2021 metais.  

3. Atvaizduoti juridinių asmenų formos ir statuso pasiskirstymą kiekvienais metais. 

4. Kiekvienai įmonei apskaičiuoti tendenciją. Jei “grynasis_pelnas” + “nuosavas_kapitalas” 2021 metais buvo didesnis už 2020 metais deklaruotą, tai tendencija “teigiama”, jei ne - “neigiama”. Atvaizduoti tendencijos duomenis grafiškai. 

5. Surasti įmones, kurios 2020 vykdė veiklą, o 2021 - bankrutavo. Bankrutavusių įmonių statusai: “Bankrutavęs”, “Bankrutuojantis”, “Išregistruotas”, “Inicijuojamas likvidavimas”, “Likviduojamas”, “Likviduojamas dėl bankroto”. Kokia bankrutavusių įmonių “nuosavas_kapitalas” suma ir “grynasis_pelnas” suma 2021 metais? Palyginti šias sumas su veikiančių įmonių atitinkamomis sumomis. 

6. Sukurti naują stulpelį “turtas”, kuris skaičiuojamas sudedant “trumpalaikis_turtas” ir “ilgalaikis_turtas”. 

7. Suskirstyti įmones į 5 grupes pagal lauką “turtas”. Palyginti kaip šios grupės keičiasi 2020 ir 2021 metais. Palyginimą atvaizduoti grafiškai. 

8. Surasti top100 pelningiausias įmones pagal “grynasis pelnas” ir top100 įmones kurių “nuosavas kapitalas” didžiausias. Ar yra sutampančių įmonių šiuose sąrašuose? Skaičiuojant topus imti viso laikotarpio įmones. 

9. Ar topai iš 9 užduoties skiriasi 2020 ir 2021 metais? Kurios naujos iškrenta iš topų 2021 metais? 

In [4]:
import pandas as pd

<IPython.core.display.Javascript object>

In [13]:
## got an error while downloading online csv data. OSX
## quick fix
## https://stackoverflow.com/questions/50236117/scraping-ssl-certificate-verify-failed-error-for-http-en-wikipedia-org

import ssl

ssl._create_default_https_context = ssl._create_unverified_context

<IPython.core.display.Javascript object>

# 1 
Parsisiųsti 2020 ir 2021 įmonių “Juridinių asmenų pateikti finansinės atskaitomybės dokumentai – balanso ataskaitos” ir “Juridinių asmenų pateikti finansinės atskaitomybės dokumentai – pelno (nuostolių) ataskaitos” duomenis.

In [11]:
balance_20 = pd.read_csv(
    "https://www.registrucentras.lt/aduomenys/?byla=JAR_FA_RODIKLIAI_BLNS_2020.csv"
)
balance_21 = pd.read_csv(
    "https://www.registrucentras.lt/aduomenys/?byla=JAR_FA_RODIKLIAI_BLNS_2021.csv"
)
pl_20 = pd.read_csv(
    "https://www.registrucentras.lt/aduomenys/?byla=JAR_FA_RODIKLIAI_PLNA_2020.csv"
)
pl_21 = pd.read_csv(
    "https://www.registrucentras.lt/aduomenys/?byla=JAR_FA_RODIKLIAI_PLNA_2021.csv"
)

<IPython.core.display.Javascript object>

In [15]:
balance_20.shape, balance_21.shape, pl_20.shape, pl_21.shape

((106318, 18), (120850, 18), (96616, 17), (110227, 17))

<IPython.core.display.Javascript object>

# 2 
Apskaičiuoti vidutinias reikšmes “pelnas_pries_apmokestinima” ir “nuosavas_kapitalas” laukų kiekvienai juridinio asmens formai (Uždaroji akcinė bendrovė, Mažoji bendrija, etc.) už visą laikotarpį.

In [24]:
balance_20.form_pav.value_counts(dropna=False)

Uždaroji akcinė bendrovė                    76370
Mažoji bendrija                             13196
Asociacija                                   9073
Viešoji įstaiga                              5324
Individuali įmonė                             677
Labdaros ir paramos fondas                    598
Žemės ūkio bendrovė                           401
Kooperatinė bendrovė                          334
Akcinė bendrovė                               245
Komanditinė ūkinė bendrija                     39
Savivaldybės įmonė                             24
Užsienio juridinio asmens filialas             16
Valstybės įmonė                                13
Tikroji ūkinė bendrija                          4
Biudžetinė įstaiga                              2
Bendrija                                        1
Lietuvos prekyb.,pramon. rūmų asociacija        1
Name: form_pav, dtype: int64

<IPython.core.display.Javascript object>

In [26]:
# nuosavas_kapitalas
nuosavas_kapitalas = balance_20[["obj_kodas", "nuosavas_kapitalas"]]

<IPython.core.display.Javascript object>

In [29]:
nuosavas_kapitalas.shape

(106318, 2)

<IPython.core.display.Javascript object>

In [27]:
# form_pav,  pelnas_pries_apmokestinima
pelnas_pries_apmokestinima = pl_20[
    ["obj_kodas", "form_pav", "pelnas_pries_apmokestinima"]
]

<IPython.core.display.Javascript object>

In [30]:
pelnas_pries_apmokestinima.shape

(96616, 3)

<IPython.core.display.Javascript object>

In [35]:
balance_20_set = set(balance_20.obj_kodas)
pl_20_set = set(pl_20.obj_kodas)

len(pl_20_set.difference(balance_20_set)), len(balance_20_set.difference(pl_20_set))

(0, 8120)

<IPython.core.display.Javascript object>

In [43]:
# we take intersection and ignore companies that do not have profit loss statement.
pelnas_pries_apmokestinima.merge(nuosavas_kapitalas, on="obj_kodas", how="inner").shape

(129210, 4)

<IPython.core.display.Javascript object>

In [39]:
# we see there are duplicates !!!!

(96616, 3)

<IPython.core.display.Javascript object>

In [52]:
balance_20.obj_kodas.value_counts().iloc[:3]

168970870    16
145496790    16
135994223    15
Name: obj_kodas, dtype: int64

<IPython.core.display.Javascript object>

In [70]:
balance_20[balance_20.laikotarpis_nuo == "2019-01-01"].obj_kodas.value_counts()

304127368    1
304895492    1
302583362    1
303445421    1
124665536    1
            ..
177218596    1
151484255    1
125416121    1
302756355    1
303778867    1
Name: obj_kodas, Length: 86828, dtype: int64

<IPython.core.display.Javascript object>

In [77]:
# we filter out duplicates(?) by choosing laikotarpis_nuo == "2019-01-01". This requires inside knowledge about balance sheets.
nuosavas_kapitalas = balance_20[balance_20.laikotarpis_nuo == "2019-01-01"][
    ["obj_kodas", "nuosavas_kapitalas"]
]

<IPython.core.display.Javascript object>

In [79]:
pelnas_pries_apmokestinima = pl_20[pl_20.laikotarpis_nuo == "2019-01-01"][
    ["obj_kodas", "form_pav", "pelnas_pries_apmokestinima"]
]

<IPython.core.display.Javascript object>

In [75]:
pelnas_pries_apmokestinima.obj_kodas.value_counts() ## no duplicates. 

304781777    1
111733617    1
302794270    1
302664106    1
300506602    1
            ..
110461363    1
303336094    1
304040760    1
300572972    1
173110025    1
Name: obj_kodas, Length: 79433, dtype: int64

<IPython.core.display.Javascript object>

In [80]:
nuosavas_kapitalas.shape, pelnas_pries_apmokestinima.shape

((86828, 2), (79433, 3))

<IPython.core.display.Javascript object>

In [87]:
# we drop nan and use inner merge
merged = nuosavas_kapitalas.merge(
    pelnas_pries_apmokestinima, on="obj_kodas", how="inner"
)

<IPython.core.display.Javascript object>

In [103]:
year_20_result = merged.groupby('form_pav').agg({'obj_kodas':'count', 
                         'nuosavas_kapitalas':'mean', 
                         'pelnas_pries_apmokestinima':'mean', }).apply(round).rename(columns=dict(obj_kodas='count'))


<IPython.core.display.Javascript object>

In [104]:
year_20_result

,count,nuosavas_kapitalas,pelnas_pries_apmokestinima
form_pav,,,
Akcinė bendrovė,234,38071183.0,2949222.0
Asociacija,1984,15481.0,65.0
Biudžetinė įstaiga,2,141074.0,16242.0
Individuali įmonė,562,191930.0,19217.0
Komanditinė ūkinė bendrija,23,1442068.0,711898.0
Kooperatinė bendrovė,317,698003.0,36818.0
Labdaros ir paramos fondas,91,10945.0,141.0
"Lietuvos prekyb.,pramon. rūmų asociacija",1,64491.0,NaN
Mažoji bendrija,8562,18645.0,7669.0


<IPython.core.display.Javascript object>

In [109]:
## year 2021

<IPython.core.display.Javascript object>

In [110]:
nuosavas_kapitalas = balance_21[balance_21.laikotarpis_nuo == "2019-01-01"][
    ["obj_kodas", "nuosavas_kapitalas"]
]
pelnas_pries_apmokestinima = pl_21[pl_21.laikotarpis_nuo == "2019-01-01"][
    ["obj_kodas", "form_pav", "pelnas_pries_apmokestinima"]
]
## check for duplicates.
nuosavas_kapitalas.shape, pelnas_pries_apmokestinima.shape, len(
    set(nuosavas_kapitalas.obj_kodas)
), len(set(pelnas_pries_apmokestinima.obj_kodas))
#  No duplicates!!

((9191, 2), (8185, 3), 9191, 8185)

<IPython.core.display.Javascript object>

In [107]:
merged = nuosavas_kapitalas.merge(
    pelnas_pries_apmokestinima, on="obj_kodas", how="inner"
)
year_21_result = merged.groupby('form_pav').agg({'obj_kodas':'count', 
                         'nuosavas_kapitalas':'mean', 
                         'pelnas_pries_apmokestinima':'mean', }).apply(round).rename(columns=dict(obj_kodas='count'))


<IPython.core.display.Javascript object>

In [108]:
year_21_result

,count,nuosavas_kapitalas,pelnas_pries_apmokestinima
form_pav,,,
Akcinė bendrovė,3,2015392.0,187988.0
Asociacija,240,866620.0,-145.0
Individuali įmonė,719,60998.0,13395.0
Komanditinė ūkinė bendrija,7,7443434.0,250916.0
Kooperatinė bendrovė,18,-136089.0,-64649.0
Labdaros ir paramos fondas,13,-15291.0,7476.0
Mažoji bendrija,1224,12777.0,4451.0
Savivaldybės įmonė,1,2939980.0,-65242.0
Tikroji ūkinė bendrija,8,307520.0,19271.0


<IPython.core.display.Javascript object>

In [ ]:
## additional task -> write a function do not repeat the code. 

In [ ]:
## additional tasks
# 1. add to a single table that holds hieararchical structure. 

In [118]:
merged_result = pd.concat({"2020": year_20_result, "2021": year_21_result}, axis=1)

<IPython.core.display.Javascript object>

In [123]:
merged_result

2020                     \
                                          count nuosavas_kapitalas   
form_pav                                                             
Akcinė bendrovė                             234         38071183.0   
Asociacija                                 1984            15481.0   
Biudžetinė įstaiga                            2           141074.0   
Individuali įmonė                           562           191930.0   
Komanditinė ūkinė bendrija                   23          1442068.0   
Kooperatinė bendrovė                        317           698003.0   
Labdaros ir paramos fondas                   91            10945.0   
Lietuvos prekyb.,pramon. rūmų asociacija      1            64491.0   
Mažoji bendrija                            8562            18645.0   
Savivaldybės įmonė                           18          3891527.0   
Tikroji ūkinė bendrija                        4           470602.0   
Uždaroji akcinė bendrovė                  64401           582191.0   
Užsienio juridinio asmens filialas           13          -296372.0   
Valstybės įmonė                              13         94670868.0   
Viešoji įstaiga                            2868            36791.0   
Žemės ūkio bendrovė                         340          1497443.0   

                                                                       2021  \
                                         pelnas_pries_apmokestinima   count   
form_pav                                                                      
Akcinė bendrovė                                           2949222.0     3.0   
Asociacija                                                     65.0   240.0   
Biudžetinė įstaiga                                          16242.0     NaN   
Individuali įmonė                                           19217.0   719.0   
Komanditinė ūkinė bendrija                                 711898.0     7.0   
Kooperatinė bendrovė                                        36818.0    18.0   
Labdaros ir paramos fondas                                    141.0    13.0   
Lietuvos prekyb.,pramon. rūmų asociacija                        NaN     NaN   
Mažoji bendrija                                              7669.0  1224.0   
Savivaldybės įmonė                                          86824.0     1.0   
Tikroji ūkinė bendrija                                    -103957.0     8.0   
Uždaroji akcinė bendrovė                                   109280.0  5538.0   
Užsienio juridinio asmens filialas                          98792.0     2.0   
Valstybės įmonė                                           5804726.0     1.0   
Viešoji įstaiga                                              9029.0   381.0   
Žemės ūkio bendrovė                                         44135.0    30.0   

                                                             \
                                         nuosavas_kapitalas   
form_pav                                                      
Akcinė bendrovė                                   2015392.0   
Asociacija                                         866620.0   
Biudžetinė įstaiga                                      NaN   
Individuali įmonė                                   60998.0   
Komanditinė ūkinė bendrija                        7443434.0   
Kooperatinė bendrovė                              -136089.0   
Labdaros ir paramos fondas                         -15291.0   
Lietuvos prekyb.,pramon. rūmų asociacija                NaN   
Mažoji bendrija                                     12777.0   
Savivaldybės įmonė                                2939980.0   
Tikroji ūkinė bendrija                             307520.0   
Uždaroji akcinė bendrovė                           120752.0   
Užsienio juridinio asmens filialas                    864.0   
Valstybės įmonė                                   4369323.0   
Viešoji įstaiga                                     23551.0   
Žemės ūkio bendrovė                                745970.0   

            

<IPython.core.display.Javascript object>

# 3
Atrinkti įmones, kurių “nuosavas_kapitalas” buvo didesnis negu nuosavo kapitalo mediana 2020 ir 2021 metais.